In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report as cr
import pickle

warnings.simplefilter(action = 'ignore', category = FutureWarning)
k_folds = KFold(n_splits = 10)

In [2]:
dataset = pd.read_csv("News_dataset_preprocessed.csv")
dataset.drop(dataset.tail(39898).index, inplace = True)
articles = [w for w in dataset["text"]]

In [3]:
pickle_in = open("fasttext_vectors.pickle", "rb")
word_vectors_fastText = pickle.load(pickle_in)
del word_vectors_fastText[-39898:]

In [4]:
def getArticleVector(article):
    return np.mean([word for word in article], axis = 0)

In [5]:
x = []
for i in range(len(word_vectors_fastText)):
    x.append(getArticleVector(word_vectors_fastText[i]))

In [6]:
y = dataset["true"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [8]:
rfc = RandomForestClassifier(n_estimators = 100)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [9]:
score = accuracy_score(y_test, y_pred)
print(score)
print(cr(y_test, y_pred))

0.949
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       515
           1       0.94      0.95      0.95       485

    accuracy                           0.95      1000
   macro avg       0.95      0.95      0.95      1000
weighted avg       0.95      0.95      0.95      1000



In [10]:
# 10-fold cross-validation
scores = cross_val_score(rfc, x, y, cv = k_folds)

In [11]:
# 10-fold cross-validation results
print("CV scores: ", scores)
print("Avg CV score: ", scores.mean())

CV scores:  [0.946 0.936 0.954 0.938 0.956 0.95  0.942 0.946 0.95  0.954]
Avg CV score:  0.9471999999999999
